In [46]:
import re
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [116]:
def search_and_replace(s):
    # Converts 
    # s = 'sc mc fc( hcj( + sb* ) ) hcj*( fc*( + sc mc ) )'
    # to 
    # {sc} {mc} {fc}( {hcj}( + {sb}* ) ) {hcj}*( {fc}*( + {sc} {mc} ) )
    matches = re.findall('[a-z]+', s)
    matches = list(set(matches))
    
    matches.sort(key= lambda x: len(x), reverse=True)
    for match in matches:
        rep = '{' + match + '}'
        rep = rep.upper()
        s = re.sub(match, rep, s)
    return s.lower()

s = 'mcr(scr( sc( mc( + fc* ) ) ) ) fcr*'
print(search_and_replace(s))

{mcr}({scr}( {sc}( {mc}( + {fc}* ) ) ) ) {fcr}*


In [55]:
# length hcj = 15
# length hck = 15
# length hbr = 15
# length sb = 7
# length mb = 15
# length sc = 7
# length fc = 7
# length sc = 7
# length fb = 7
# length mc = 15

# length hcjr = 15
# length hckr = 15
# length hbrr = 15
# length sbr = 7
# length mbr = 15
# length scr = 7
# length fcr = 7
# length scr = 7
# length fbr = 7
# length mcr = 15


# Br = sb mb fb hbr @initial 10 nM
# Cj = sc mc fc hcj @initial 10 nM

# ReactCBCj = fb(sc( mc( + fc* ) ) ) mb*(sb*( + hcj) ) @initial 100 nM
# BackCB = fb sc mc @initial 100 nM
# ProduceBCjCj = sc mc fc( hcj( + sb* ) ) hcj*( fc*( + sc mc ) ) @initial 100 nM
# HelperCCj = fc hcj fc @initial 75 nM



# sBr = hbrr fbr mbr sbr @initial 10 nM
# sCj = hcjr fcr mcr scr @initial 1e-4 nM

# sReactCBCj = mbr( sbr( hcjr + ) ) fbr*( scr*( mcr*( fcr* + ) ) ) @initial 100 nM
# sBackCB = mcr scr fbr @initial 100 nM
# sProduceBCjCj = hcjr( fcr( mcr scr + hcjr( fcr( mcr scr + ) ) ) ) sbr* @initial 100 nM
# sHelperCCj = fcr hcjr fcr @initial 75 nM


# LeakWaste = sc mc fc hcj( + sb* ) fc*( hcj*( fc*( + ) ) ) @initial 0 nM
# sLeajWaste = hcjr( fcr mcr scr + fcr( hcjr( fcr( + ) ) ) ) sbr* @initial 0 nM

# Cancel = mcr( scr( sc( mc( + fc* ) ) ) ) fcr* @initial 100 nM
# disp = mcr scr sc mc @initial 0 nM         

In [50]:
def reverse(domain):
    return domain + 'R'

def shadow(domain):
    return 's' + domain

def create_domains(signal='C', 
                   subsignal='j', 
                   history_len=15, 
                   migration_len=15, 
                   toehold_len=7):
    domains = {}
    
    history_domain = 'h' + signal + subsignal
    rev_history_domain = reverse(history_domain)
    
    ftoehold_domain = 'f' + signal 
    rev_ftoehold_domain = reverse(ftoehold_domain)
    stoehold_domain = 's' + signal
    rev_stoehold_domain = reverse(stoehold_domain)
    migration_domain = 'm' + signal
    rev_migration_domain = reverse(migration_domain)
    
    domains['history'] = (history_domain, history_len)
    domains['ftoehold'] = (ftoehold_domain, toehold_len)
    domains['stoehold'] = (stoehold_domain, toehold_len)
    domains['migration'] = (migration_domain, migration_len)
    domains['rev_history'] = (rev_history_domain, history_len)
    domains['rev_ftoehold'] = (rev_ftoehold_domain, toehold_len)
    domains['rev_stoehold'] = (rev_stoehold_domain, toehold_len)
    domains['rev_migration'] = (rev_migration_domain, migration_len)
    
    return domains

In [77]:
print(search_and_replace('hcjr( fcr( mcr scr + hcjr( fcr( mcr scr + ) ) ) ) sbr*'))

{hcjr}( {fcr}( {mcr} {scr} + {hcjr}( {fcr}( {mcr} {scr} + ) ) ) ) {sbr}*


In [125]:
def construct_signal_strands(signal, subsignal, initial=10, shadow=False):
    name = signal + subsignal
    ads = create_domains(signal, subsignal)
    
    if not shadow:    
        # Cj = sc mc fc hcj @initial 10 nM
        fc = ads['ftoehold'][0]
        sc = ads['stoehold'][0]
        mc = ads['migration'][0]
        hcj = ads['history'][0]
        structure = f'{sc} {mc} {fc} {hcj} @initial {initial} nM'
    else:
        name = 's' + name
        fcr = ads['rev_ftoehold'][0]
        scr = ads['rev_stoehold'][0]
        mcr = ads['rev_migration'][0]
        hcjr = ads['rev_history'][0]
        structure = f'{hcjr} {fcr} {mcr} {scr} @initial {initial} nM'
    return name, structure
    
def construct_react_complex(amp, subamp, fuel, subfuel, initial=100, shadow=False):
    
    name = 'React' + amp + fuel + amp + subamp
    ads = create_domains(signal=amp,
                             subsignal=subamp)
    fds = create_domains(signal=fuel,
                         subsignal=subfuel)
    if not shadow:
        
        # These are proxies for easy reproduction
        fb = fds['ftoehold'][0] 
        sb = fds['stoehold'][0]
        mb = fds['migration'][0]
        hbr = fds['history'][0]

        fc = ads['ftoehold'][0]
        sc = ads['stoehold'][0]
        mc = ads['migration'][0]
        hcj = ads['history'][0]

        structure = f"{fb}( {sc}( {mc}( + {fc}* ) ) ) {mb}*( {sb}*( + {hcj} ) ) @initial {initial} nM"
   
    else:
        name = 's' + name
        fbr = fds['rev_ftoehold'][0] 
        sbr = fds['rev_stoehold'][0]
        mbr = fds['rev_migration'][0]
        hbrr = fds['rev_history'][0]

        fcr = ads['rev_ftoehold'][0]
        scr = ads['rev_stoehold'][0]
        mcr = ads['rev_migration'][0]
        hcjr = ads['rev_history'][0]
        
        structure = f"{mbr}( {sbr}( {hcjr} + ) ) {fbr}*( {scr}*( {mcr}*( {fcr}* + ) ) ) @initial {initial} nM"
    
    return name, structure

def construct_back_strand(amp, subamp, fuel, subfuel, initial=100, shadow=False):

    name = 'Back' + amp + fuel
    ads = create_domains(signal=amp, subsignal=subamp)
    fds = create_domains(signal=fuel, subsignal=subfuel)
    if not shadow:
        
        # These are proxies for easy reproduction
        fb = fds['ftoehold'][0]
        sb = fds['stoehold'][0]
        mb = fds['migration'][0]
        hbr = fds['history'][0]

        fc = ads['ftoehold'][0]
        sc = ads['stoehold'][0]
        mc = ads['migration'][0]
        hcj = ads['history'][0]

        structure = f"{fb} {sc} {mc} @initial {initial} nM"
    else:
        name = 's' + name
        fbr = fds['rev_ftoehold'][0] 
        sbr = fds['rev_stoehold'][0]
        mbr = fds['rev_migration'][0]
        hbrr = fds['rev_history'][0]

        fcr = ads['rev_ftoehold'][0]
        scr = ads['rev_stoehold'][0]
        mcr = ads['rev_migration'][0]
        hcjr = ads['rev_history'][0]
        
        structure = f"{mcr} {scr} {fbr} @initial {initial} nM"
        
    return name, structure 

def construct_produce_complex(amp, subamp, fuel, subfuel, initial=100, shadow=False):
    # ProduceBCjCj = sc mc fc( hcj( + sb* ) ) hcj*( fc*( + sc mc ) ) @initial 100 nM
    
    name = 'Produce' + fuel + amp + subamp + amp + subamp
    ads = create_domains(signal=amp,
                             subsignal=subamp)
    fds = create_domains(signal=fuel,
                         subsignal=subfuel)
    if not shadow:
        # These are proxies for easy reproduction
        fb = fds['ftoehold'][0]
        sb = fds['stoehold'][0]
        mb = fds['migration'][0]
        hbr = fds['history'][0]

        fc = ads['ftoehold'][0]
        sc = ads['stoehold'][0]
        mc = ads['migration'][0]
        hcj = ads['history'][0]
    
        structure = f"{sc} {mc} {fc}( {hcj}( + {sb}* ) ) {hcj}*( {fc}*( + {sc} {mc} ) ) @initial {initial} nM"
    else:
        name = 's' + name
        fbr = fds['rev_ftoehold'][0] 
        sbr = fds['rev_stoehold'][0]
        mbr = fds['rev_migration'][0]
        hbrr = fds['rev_history'][0]

        fcr = ads['rev_ftoehold'][0]
        scr = ads['rev_stoehold'][0]
        mcr = ads['rev_migration'][0]
        hcjr = ads['rev_history'][0]
        
        structure = f"{hcjr}( {fcr}( {mcr} {scr} + {hcjr}( {fcr}( {mcr} {scr} + ) ) ) ) {sbr}* @initial " + \
                                                                                            f"{initial} nM"
        
    return name, structure 

def construct_helper_strand(amp, subamp, fuel, subfuel, initial=75, shadow=False):
    # Helper : fc hcj fc @initial 75 nM
    name = 'Helper' + amp +  amp + subamp
    ads = create_domains(signal=amp,
                                subsignal=subamp)
    fds = create_domains(signal=fuel,
                                 subsignal=subfuel)
    if not shadow:
        # These are proxies for easy reproduction
        fb = fds['ftoehold'][0]
        sb = fds['stoehold'][0]
        mb = fds['migration'][0]
        hbr = fds['history'][0]

        fc = ads['ftoehold'][0]
        sc = ads['stoehold'][0]
        mc = ads['migration'][0]
        hcj = ads['history'][0]

        structure = f"{fc} {hcj} {fc} @initial {initial} nM"
    else:
        name = 's' + name
        fbr = fds['rev_ftoehold'][0] 
        sbr = fds['rev_stoehold'][0]
        mbr = fds['rev_migration'][0]
        hbrr = fds['rev_history'][0]

        fcr = ads['rev_ftoehold'][0]
        scr = ads['rev_stoehold'][0]
        mcr = ads['rev_migration'][0]
        hcjr = ads['rev_history'][0]
        
        structure = f"{fcr} {hcjr} {fcr} @initial {initial} nM"
    
    return name, structure 

def construct_produce_helper_leak(amp, subamp, fuel, subfuel, initial=0, shadow=False):
    # LeakWaste = sc mc fc hcj( + sb* ) fc*( hcj*( fc*( + ) ) ) @initial 0 nM
    name = 'LeakWaste' + amp + fuel
    ads = create_domains(signal=amp,
                                subsignal=subamp)
    fds = create_domains(signal=fuel,
                                 subsignal=subfuel)
    if not shadow:
        # These are proxies for easy reproduction
        fb = fds['ftoehold'][0]
        sb = fds['stoehold'][0]
        mb = fds['migration'][0]
        hbr = fds['history'][0]

        fc = ads['ftoehold'][0]
        sc = ads['stoehold'][0]
        mc = ads['migration'][0]
        hcj = ads['history'][0]
    
        structure = f"{sc} {mc} {fc} {hcj}( + {sb}* ) {fc}*( {hcj}*( {fc}*( + ) ) ) @initial {initial} nM"
    else:
        name = 's' + name
        fbr = fds['rev_ftoehold'][0] 
        sbr = fds['rev_stoehold'][0]
        mbr = fds['rev_migration'][0]
        hbrr = fds['rev_history'][0]

        fcr = ads['rev_ftoehold'][0]
        scr = ads['rev_stoehold'][0]
        mcr = ads['rev_migration'][0]
        hcjr = ads['rev_history'][0]
        
        structure = f"{hcjr}( {fcr} {mcr} {scr} + {fcr}( {hcjr}( {fcr}( + ) ) ) ) {sbr}* @initial {initial} nM"
    return name, structure

def construct_cancel_complex(amp, subamp, fuel, subfuel, initial=0, shadow=False):
    name = 'Cancel' + amp
    ads = create_domains(signal=amp,
                                subsignal=subamp)
    fds = create_domains(signal=fuel,
                                 subsignal=subfuel)
    fc = ads['ftoehold'][0]
    sc = ads['stoehold'][0]
    mc = ads['migration'][0]
    hcj = ads['history'][0]

    fcr = ads['rev_ftoehold'][0]
    scr = ads['rev_stoehold'][0]
    mcr = ads['rev_migration'][0]
    hcjr = ads['rev_history'][0]

    structure = f"{mcr}( {scr}( {sc}( {mc}( + {fc}* ) ) ) ) {fcr}* @initial {initial} nM"
    return name, structure

def construct_disp_strand(amp, subamp, fuel, subfuel, initial=0, shadow=False):
    name = 'disp' + amp
    ads = create_domains(signal=amp,
                                subsignal=subamp)
    fds = create_domains(signal=fuel,
                                 subsignal=subfuel)
    fc = ads['ftoehold'][0]
    sc = ads['stoehold'][0]
    mc = ads['migration'][0]
    hcj = ads['history'][0]

    fcr = ads['rev_ftoehold'][0]
    scr = ads['rev_stoehold'][0]
    mcr = ads['rev_migration'][0]
    hcjr = ads['rev_history'][0]
    
    structure = f"{mcr} {scr} {sc} {mc} @initial 0 nM"
    return name, structure

def construct_cancel_waste_complex(amp, subamp, fuel, subfuel, initial=0, shadow=False):
    
    name = 'CancelWaste' + amp
    ads = create_domains(signal=amp,
                                subsignal=subamp)
    fds = create_domains(signal=fuel,
                                 subsignal=subfuel)
    fc = ads['ftoehold'][0]
    sc = ads['stoehold'][0]
    mc = ads['migration'][0]
    hcj = ads['history'][0]

    fcr = ads['rev_ftoehold'][0]
    scr = ads['rev_stoehold'][0]
    mcr = ads['rev_migration'][0]
    hcjr = ads['rev_history'][0]
    
    structure = f"{sc}( {mc}( {fc}( {hcj} + ) ) ) {scr}*( {mcr}*( {fcr}*( + {hcjr} ) ) ) @initial {initial} nM"
    return name, structure

In [126]:
print(search_and_replace('mcr( scr( sc( mc( + fc* ) ) ) ) fcr*'))

{mcr}( {scr}( {sc}( {mc}( + {fc}* ) ) ) ) {fcr}*


In [128]:

def biamp(amp='C', subamp='j', fuel='B', subfuel='r', leak=False, shadow=False):
    """
    Creates DNA Produce-Helper style reactions for Cj + Br --> Cj + Cj
    """
    ads = create_domains(amp, subamp)
    fds = create_domains(fuel, subfuel)
    Cjname, Cj = construct_signal_strands(amp, subamp, initial=10, shadow=shadow)
    Brname, Br = construct_signal_strands(fuel, subfuel, initial=10, shadow=shadow)
    
    for k, v in ads.items():
        print(f'length {v[0]} = {v[1]}')
    
    for k, v in fds.items():
        print(f'length {v[0]} = {v[1]}')
    
    print(f'{Cjname} = {Cj}')
    print(f'{Brname} = {Br}')
    
    complexes = [
        construct_react_complex(amp, subamp, fuel, subfuel, shadow=shadow), 
        construct_produce_complex(amp, subamp, fuel, subfuel, shadow=shadow),
        construct_back_strand(amp, subamp, fuel, subfuel, shadow=shadow),
        construct_helper_strand(amp, subamp, fuel, subfuel, shadow=shadow),
        construct_cancel_complex(amp, subamp, fuel, subfuel, shadow=shadow),
        construct_disp_strand(amp, subamp, fuel, subfuel, shadow=shadow),
        construct_cancel_waste_complex(amp, subamp, fuel, subfuel, initial=0, shadow=shadow)
    ]
    for name, structure in complexes:
        print(f'{name} = {structure}')
    
    if leak:
        leak_name, leak_struct = construct_produce_helper_leak(amp, subamp, fuel, subfuel, shadow=shadow)
        print(f'{leak_name} = {leak_struct}')
        
    
        

biamp(amp='A', subamp='k', fuel='C', subfuel='j', leak=False, shadow=True)

length hAk = 15
length fA = 7
length sA = 7
length mA = 15
length hAkR = 15
length fAR = 7
length sAR = 7
length mAR = 15
length hCj = 15
length fC = 7
length sC = 7
length mC = 15
length hCjR = 15
length fCR = 7
length sCR = 7
length mCR = 15
sAk = hAkR fAR mAR sAR @initial 10 nM
sCj = hCjR fCR mCR sCR @initial 10 nM
sReactACAk = mCR( sCR( hAkR + ) ) fCR*( sAR*( mAR*( fAR* + ) ) ) @initial 100 nM
sProduceCAkAk = hAkR( fAR( mAR sAR + hAkR( fAR( mAR sAR + ) ) ) ) sCR* @initial 100 nM
sBackAC = mAR sAR fCR @initial 100 nM
sHelperAAk = fAR hAkR fAR @initial 75 nM
CancelA = mAR( sAR( sA( mA( + fA* ) ) ) ) fAR* @initial 0 nM
dispA = mAR sAR sA mA @initial 0 nM
CancelWasteA = sA( mA( fA( hAk + ) ) ) sAR*( mAR*( fAR*( + hAkR ) ) ) @initial 0 nM


In [129]:
shadow = True
leak = True
biamp(amp='C', subamp='j', fuel='B', subfuel='r', leak=leak, shadow=shadow)
biamp(amp='B', subamp='r', fuel='A', subfuel='k', leak=leak, shadow=shadow)
biamp(amp='A', subamp='k', fuel='C', subfuel='j', leak=leak, shadow=shadow)


length hCj = 15
length fC = 7
length sC = 7
length mC = 15
length hCjR = 15
length fCR = 7
length sCR = 7
length mCR = 15
length hBr = 15
length fB = 7
length sB = 7
length mB = 15
length hBrR = 15
length fBR = 7
length sBR = 7
length mBR = 15
sCj = hCjR fCR mCR sCR @initial 10 nM
sBr = hBrR fBR mBR sBR @initial 10 nM
sReactCBCj = mBR( sBR( hCjR + ) ) fBR*( sCR*( mCR*( fCR* + ) ) ) @initial 100 nM
sProduceBCjCj = hCjR( fCR( mCR sCR + hCjR( fCR( mCR sCR + ) ) ) ) sBR* @initial 100 nM
sBackCB = mCR sCR fBR @initial 100 nM
sHelperCCj = fCR hCjR fCR @initial 75 nM
CancelC = mCR( sCR( sC( mC( + fC* ) ) ) ) fCR* @initial 0 nM
dispC = mCR sCR sC mC @initial 0 nM
CancelWasteC = sC( mC( fC( hCj + ) ) ) sCR*( mCR*( fCR*( + hCjR ) ) ) @initial 0 nM
sLeakWasteCB = hCjR( fCR mCR sCR + fCR( hCjR( fCR( + ) ) ) ) sBR* @initial 0 nM
length hBr = 15
length fB = 7
length sB = 7
length mB = 15
length hBrR = 15
length fBR = 7
length sBR = 7
length mBR = 15
length hAk = 15
length fA = 7
length sA = 7
length

In [130]:
print(search_and_replace('hcjr fcr( mcr( scr( + mcr scr sc( mc( + fc* ) ) ) ) )'))

{hcjr} {fcr}( {mcr}( {scr}( + {mcr} {scr} {sc}( {mc}( + {fc}* ) ) ) ) )


In [150]:
s = ' hcjr fcr( mcr( scr( + mcr scr sc( mc( + fc* ) ) ) ) )'
print(search_and_replace(s))

 {hcjr} {fcr}( {mcr}( {scr}( + {mcr} {scr} {sc}( {mc}( + {fc}* ) ) ) ) )


In [151]:
def generic(s, amp, subamp, fuel, subfuel, initial=0):
    ads = create_domains(signal=amp,
                                subsignal=subamp)
    fds = create_domains(signal=fuel,
                                 subsignal=subfuel)

    # These are proxies for easy reproduction
    fb = fds['ftoehold'][0]
    sb = fds['stoehold'][0]
    mb = fds['migration'][0]
    hbr = fds['history'][0]

    fc = ads['ftoehold'][0]
    sc = ads['stoehold'][0]
    mc = ads['migration'][0]
    hcj = ads['history'][0]

    fbr = fds['rev_ftoehold'][0] 
    sbr = fds['rev_stoehold'][0]
    mbr = fds['rev_migration'][0]
    hbrr = fds['rev_history'][0]

    fcr = ads['rev_ftoehold'][0]
    scr = ads['rev_stoehold'][0]
    mcr = ads['rev_migration'][0]
    hcjr = ads['rev_history'][0]
    structure = f" {hcjr} {fcr}( {mcr}( {scr}( + {mcr} {scr} {sc}( {mc}( + {fc}* ) ) ) ) )"
    return structure
print(generic(s, amp='A', subamp='k', fuel='B', subfuel='r'))

 hAkR fAR( mAR( sAR( + mAR sAR sA( mA( + fA* ) ) ) ) )


'hCjR fCR( mCR( sCR( + mCR sCR sC( mC( + fC* ) ) ) ) )'